In [1]:
!pip install transformers datasets torch scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset, Dataset
from sklearn.metrics import classification_report
import random

In [3]:
# Load a pre-trained spam filter model
MODEL_NAME = "mariagrandury/distilbert-base-uncased-finetuned-sms-spam-detection"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [4]:
# Load the SMS Spam Dataset
dataset = load_dataset("sms_spam")

# Manually split the dataset (80% train, 20% test)
train_size = int(0.8 * len(dataset["train"]))
train_dataset, test_dataset = dataset["train"].select(range(train_size)), dataset["train"].select(range(train_size, len(dataset["train"])))

# Display a sample
print("Sample Data:")
print(train_dataset[0])

README.md:   0%|          | 0.00/4.98k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/359k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5574 [00:00<?, ? examples/s]

Sample Data:
{'sms': 'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...\n', 'label': 0}


In [5]:
# Function to tokenize data
def tokenize_data(example):
    return tokenizer(example["sms"], truncation=True, padding="max_length", max_length=128)

# Tokenize the dataset
train_dataset = train_dataset.map(tokenize_data, batched=True)
test_dataset = test_dataset.map(tokenize_data, batched=True)

# Convert datasets to PyTorch format
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

Map:   0%|          | 0/4459 [00:00<?, ? examples/s]

Map:   0%|          | 0/1115 [00:00<?, ? examples/s]

In [6]:
# Poison the training data
poisoning_ratio = 0.3  # Poison 30% of the training dataset
n_poison = int(len(train_dataset) * poisoning_ratio)
poisoned_indices = random.sample(range(len(train_dataset)), n_poison)

# Introduce poisoned samples
def poison_label(example, idx):
    if idx in poisoned_indices:
        # Flip label: 'ham' to 'spam' or vice versa
        example["label"] = 1 - example["label"]
    return example

train_dataset = train_dataset.map(poison_label, with_indices=True)

Map:   0%|          | 0/4459 [00:00<?, ? examples/s]

In [7]:
# Fine-tune the model on the poisoned dataset
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir="./logs",
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,0.273843
2,0.635800,0.348514
3,0.635800,0.363385


TrainOutput(global_step=837, training_loss=0.6218078868505917, metrics={'train_runtime': 176.5902, 'train_samples_per_second': 75.752, 'train_steps_per_second': 4.74, 'total_flos': 443004097955328.0, 'train_loss': 0.6218078868505917, 'epoch': 3.0})

In [8]:
# Evaluate the model
predictions = trainer.predict(test_dataset)
pred_labels = torch.argmax(torch.tensor(predictions.predictions), dim=1)

# Print metrics
print("Classification Report (Poisoned Model):")
print(classification_report(test_dataset["label"], pred_labels.numpy()))

Classification Report (Poisoned Model):
              precision    recall  f1-score   support

           0       1.00      0.99      0.99       970
           1       0.93      0.98      0.96       145

    accuracy                           0.99      1115
   macro avg       0.97      0.98      0.97      1115
weighted avg       0.99      0.99      0.99      1115

